In [2]:
from var4d_components import Var4D_Components
from matplotlib import pyplot as plt
from mapping_functions import MapFunctions
from scipy import optimize
import numpy as np

### Big picture

The overall objective is to minimize the cost function

$$ J = \frac{1}{2} (\mathbf{Hx} - \mathbf{z})^\text{T} \mathbf{S_z}^{-1} (\mathbf{Hx} - \mathbf{z}) + \frac{1}{2} (\mathbf{x}-\mathbf{x}_0)^\text{T} \mathbf{S_x}^{-1} (\mathbf{x}-\mathbf{x}_0) $$

"Batch" methods get the exact minimum by solving the linear equation

$$ \frac{\partial J}{\partial\mathbf{x}} = \mathbf{H}^\text{T} \mathbf{S_z}^{-1} (\mathbf{Hx} - \mathbf{z}) + \mathbf{S_x}^{-1} (\mathbf{x}-\mathbf{x}_0) = 0 $$

while variational methods get to an approximate minimum by feeding a minimizer with successive values of $J$ and $\partial J/\partial\mathbf{x}$. Which means, for every value of $\mathbf{x}$, we need a way to calculate $J$ and $\partial J\partial\mathbf{x}$. However, there is a complication.

### Ill-conditioned problems 

Consider the following function to be minimized.

$$ f(\mathbf{x}) = \sum_i \alpha_i x_i^2, \quad \alpha_i > 0 $$
$$ g(\mathbf{x}) = \frac{\partial f}{\partial\mathbf{x}} = 2\mathbf{\alpha}^\text{T} \mathbf{x} $$


In [1]:
def f(vec, prefac):
    res = 0.0
    for x,p in zip(vec, prefac):
        res = res + p*x*x
    return res
def g(vec, prefac):
    res = np.zeros(len(vec), dtype=np.float64)
    for i,(x,p) in enumerate(zip(vec,prefac)):
        res[i] = 2*p*x
    return res

We know this has a global minimum at $\mathbf{x} = \mathbf{0}$, so let's try to get that with a canned minimizer, starting from $\mathbf{x} = \mathbf{1}$ and $\alpha_i = 1$ for a vector of length $N$.

In [4]:
optim_args = dict(method='BFGS', jac=g, options=dict(maxiter=100, disp=False))
N = 20
x0 = np.ones(N) # initial guess of x
alpha = np.ones(N)
result = optimize.minimize(f, x0, args=alpha, **optim_args)

In [5]:
result.x

array([-2.77555756e-16, -3.33066907e-16, -3.33066907e-16, -1.66533454e-16,
       -4.44089210e-16, -1.11022302e-16, -1.66533454e-16, -1.11022302e-16,
       -1.11022302e-16, -1.11022302e-16,  0.00000000e+00, -1.11022302e-16,
        1.11022302e-16,  5.55111512e-17,  0.00000000e+00,  2.77555756e-16,
        4.44089210e-16,  4.44089210e-16,  4.99600361e-16,  4.99600361e-16])

Now let's make it a little more fun/non-trivial

In [8]:
optim_args = dict(method='BFGS', jac=g, options=dict(maxiter=100, disp=False))
N = 20
x0 = np.ones(N)
powers = np.random.randint(-16, 16, size=N)
alpha = 10.0**powers
result = optimize.minimize(f, x0, args=alpha, **optim_args)

In [9]:
result.x

array([ 8.26280915e-05, -1.54984390e-02, -9.70128488e-08,  1.82454236e-24,
        1.53831300e-06,  1.43347373e-11,  1.86039593e-23,  9.99887037e-01,
        5.36415891e-17,  1.85906807e-22, -1.43347707e-11,  7.09180282e-20,
        1.86808070e-20,  4.17106073e-06,  6.14084543e-04,  1.53829274e-06,
        1.86843451e-20,  9.99988704e-01,  9.99887037e-01,  1.86063315e-21])

__THINK: We all *know* that the correct solution is a vector of zeros, so what happened? Why did the minimizer not succeed?__

### Preconditioning the inversion

Instead of solving for $\mathbf{x}$, we want to solve for a vector for which the cost function is (approximately) equally "stiff" in all directions. We do this by first calculating the "square root" of the prior covariance

$$ \mathbf{L} = \sqrt{\mathbf{S_x}} $$

*Matrices don't have unique square roots*, and one possibility that is numerically efficient to compute is the Cholesky decomposition,

$$ \mathbf{S_x} = \mathbf{L}\mathbf{L}^\text{T} $$

__THINK: What are the conditions on__ $\mathbf{S_x}$ __for this to be possible?__

Once we compute $\mathbf{L}$, we define a new vector $\xi$ as $\mathbf{x} = \mathbf{L}\xi + \mathbf{x}_0$. In terms of $\xi$, the cost function and its gradient become

$$
\begin{align}
J &= \frac{1}{2} (\mathbf{HL}\xi +\mathbf{Hx}_0 - \mathbf{z})^\text{T} \mathbf{S_z}^{-1} (\mathbf{HL}\xi +\mathbf{Hx}_0 - \mathbf{z}) + \frac{1}{2} \xi^\text{T} \xi \\
\frac{\partial J}{\partial\xi} &= \mathbf{L}^\text{T}\mathbf{H}^\text{T} \mathbf{S_z}^{-1} (\mathbf{HL}\xi + \mathbf{Hx}_0 - \mathbf{z}) + \xi
\end{align}
$$

and we perform the entire optimization in terms of $\xi$, not $\mathbf{x}$.

__THINK: Why does this make the problem easier to solve, numerically?__

### Step 1: Construct $\mathbf{S_x}$

$\mathbf{S_x}$ is the covariance between prior __errors__, e.g., if the prior flux over region 1 is too high by 30% in January, what does that say about the prior flux in region 2 (next to region 1) in February? What about a region halfway around the world and six months later?

We usually model $\mathbf{S_x}$ as a function of space and time, i.e., $\mathbf{S_x}(ij) = \sigma_i \sigma_j \mathbf{R}_{ij}$, where $\sigma_i$ is the (Gaussian) error on flux element $i$ and $\mathbf{R}_{ij}$ is the correlation between prior errors of flux elements $i$ and $j$. __In this toy example__, we have further simplified $\mathbf{R}_{ij}$ to be separable in time and space, i.e., $\mathbf{R}_{ij} = \mathbf{P}_{ij} \mathbf{T}_{ij}$ where $\mathbf{P}$ and $\mathbf{T}$ and spatial and temporal correlation matrices respectively.

Finally, we've made the simplifying assumption that $\mathbf{P}_{ij} = \delta_{ij}$ and $\mathbf{T}_{ij} = e^{-\left|t_i-t_j\right|/\tau}$

In [4]:
var4d = Var4D_Components('bits_and_pieces', verbose=True, store_intermediate=True) # create an empty instance
flux_corr_structure = {'temp_corr': 2.0} # tau = 2 months
prior_corr = var4d.setup_corr(**flux_corr_structure)
var4d.prior_corr = 0.5*(prior_corr + prior_corr.T) # good idea to make it explicitly symmetric to get rid of potential asymmetries due to floating point rounding

For the toy example, we choose $\sigma_i = \beta \times \left|\text{prior flux}\right|$. This is not a great choice for fluxes that can be both positive and negative, but is often the default choice for simplicity.

In [7]:
var4d.state_prior = var4d.flux_cons.construct_state_vector_from_sib4() # prior flux will be in lat x lon x time format, need to convert to a vector
prior_unc_scale = 0.25 # beta
var4d.unc_prior = prior_unc_scale * np.abs(var4d.state_prior)

Converting SiB4 to state vector:   0%|          | 0/24 [00:00<?, ?it/s]

Now that we have both the correlation matrix $\mathbf{R}$ and the $\sigma_i$, we can make $\mathbf{S_x}$

In [8]:
prior_cov = var4d.setup_cov(var4d.prior_corr, var4d.unc_prior)
var4d.prior_cov = 0.5*(prior_cov + prior_cov.T)

Finally, construct $\mathbf{L}$ such that $\mathbf{S_x} = \mathbf{L}\mathbf{L}^\text{T}$

In [9]:
var4d.L = np.linalg.cholesky(var4d.prior_cov)

### Step 2: Construct $\mathbf{S_z}$

In this toy example, we use the same "observation errors" that were used in the OCO2 MIP. In general one needs to put in considerable thought on how to set this up. While this allows off-diagonal elements (i.e., correlation between observations), it is practically hard to implement/code. Here we will stick to a purely diagonal $\mathbf{S_z}$. We also have to make the choice of which obs to assimilate. To keep the code simple, we simply set the error of all non-assimilated obs to $10^{36}$ ppm.

In [10]:
obs_assim_dict = {'sites': ['mlo', 'spo', 'brw', 'smo']} # just the four NOAA observatories, only flask obs
var4d.obs_err = var4d.setup_obs_errors(**obs_assim_dict)

1245 of 1156383 obs will be assimilated


### Step 3: Make sure you have obs to assimilate

In the toy example, we generate the obs from a "true" flux, i.e., we assume we know reality. In real flux inversions, these are given to us by our colleagues. For illustrating the steps, we will just construct an obs vector of all 400 ppm.

In [11]:
var4d.obs_vec = 400.0 * np.ones_like(var4d.obs_err)

### Step 4: Enter the 4DVAR loop

In the 4DVAR loop, one goes from a given $\xi$ to flux ($\mathbf{x}$), transports that flux ($\mathbf{Hx}$), compares to obs ($\mathbf{Hx}-\mathbf{z}$), multiplies by $\mathbf{S_z}^{-1}$, then calculates the value of $J$ for this $\xi$. After that, the adjoint transport operator $\mathbf{H}^\text{T}$ and the adjoint preconditioning operator $\mathbf{L}^\text{T}$ are used to calculate the gradient $\partial J/\partial\xi$. This gradient and $J$ are fed to an optimizer, which gives us back the next value of $\xi$. This process repeats until some sort of convergence is reached.

#### Convert $\xi$ to flux

To illustrate the problem, let us choose a random vector $\xi$. In practice, one typically starts from $\xi = 0$.

In [12]:
xi = np.random.standard_normal(var4d.state_prior.shape[0])
x = var4d.state_prior + np.matmul(var4d.L, xi)

#### Transport this flux and sample at the observations ($\mathbf{Hx}$)

In [16]:
var4d.progress_bars = {} # stop code from complaining when I run outside a 4DVAR loop
obs = var4d.forward_transport(x)

  Added background in  0.33s
  Simulated transport in  0.44s


#### Compare to obs

In [17]:
model_obs_diff = obs - var4d.obs_vec

#### Calculate $J_\text{obs}$, where

$$
J_\text{obs} = \frac{1}{2} (\mathbf{Hx} - \mathbf{z})^\text{T} \mathbf{S_z}^{-1} (\mathbf{Hx} - \mathbf{z})
$$

In [18]:
obs_cost = 0.5 * np.sum((model_obs_diff/var4d.obs_err)**2)

#### Calculate $J_\text{bg}$ where 

$$
J_\text{bg} = \frac{1}{2} (\mathbf{x}-\mathbf{x}_0)^\text{T} \mathbf{S_x}^{-1} (\mathbf{x}-\mathbf{x}_0)
$$

and then the total cost $J = J_\text{obs}+J_\text{bg}$.

In [19]:
bg_cost = 0.5 * np.sum(xi**2)
total_cost = obs_cost + bg_cost

#### Calculate the adjoint forcing, $\mathbf{S_z}^{-1} (\mathbf{Hx} - \mathbf{z})$

In [20]:
adj_forcing = model_obs_diff/(var4d.obs_err**2)

#### Propagate this through the adjoint transport operator $\mathbf{H}^\text{T}$

In [21]:
gradient = var4d.adjoint_transport(adj_forcing)

#### Calculate the gradient $\partial J/\partial\xi$

The above us $\partial J_\text{obs}/\partial\mathbf{x}$. Convert that to $\partial J_\text{obs}/\partial\xi$,

In [22]:
gradient_preco = np.matmul(var4d.L.T, gradient)

and add the gradient of the "background", $\partial J_\text{bg}/\partial\xi = \xi$, to get the total gradient

In [23]:
total_gradient = gradient_preco + xi

The `total_cost` and `total_gradient` are then fed to an optimizer such as BFGS, which tells us the next value of $\xi$ for the next iteration.